In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [2]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [3]:
fea_number=3
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A6_001.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A6_002', 'Input_A6_003', 'Input_A6_005'], dtype=object)

## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,3)

train_rmse: 0.015629789219379738
test_rmse: 0.014470729413368688
test_r2: 0.801804072673548


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,2e-06)

test_rmse_ave: 0.014430997502281922
[0.008943306235656376, 0.013071210871812022, 0.012939423123554377, 0.01525210107549433, 0.020713636903298038, 0.010434264319936397, 0.01719925643356397, 0.016067519923427944, 0.014861325280574784, 0.014827930855501003]


test_r2_ave: 0.801730312273083
[0.8775111390545136, 0.8629461100820008, 0.8016487786139526, 0.6903473048690911, 0.8068049095825084, 0.8738216716410457, 0.7963883932699282, 0.6748465588485301, 0.8118788840826303, 0.8211093726866296]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.03],0.01)

test_rmse_ave: 0.014332522319325325
[0.007852690158250115, 0.012982532499464287, 0.012968389412896105, 0.015388599078798557, 0.021402535654027773, 0.009233184815251004, 0.01742243038328616, 0.016029697489503218, 0.015075677709783328, 0.014969485991992682]


test_r2_ave: 0.804237832582125
[0.9055640582150133, 0.8647994179465145, 0.8007597235496899, 0.684780057759734, 0.7937405639658286, 0.9011983588032993, 0.7910700678115694, 0.6763755589604344, 0.8064130153613525, 0.817677503447815]


## SVR

In [7]:
model=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,3,model)

train_rmse: 0.01679281095760865
test_rmse: 0.014703087024751349
test_r2: 0.790881695887998


## XGB

In [8]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 130, 
              max_depth = 1, 
              min_child_weight = 1,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.85,
              reg_alpha =  0,
              reg_lambda = 1)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.002472808015534328
[0.0004424669656394691, 0.0021229864794565486, 0.0014166562724227906, 0.001596696422350009, 0.0059347907807062086, 0.003341715799536615, 0.002670599807987617, 0.0022453110899450833, 0.0019153977173800795, 0.003041458819918865]


test_r2_ave: 0.9939601959677267
[0.9997001789671075, 0.9963846180869226, 0.9976224260845834, 0.9966064033741826, 0.9841403282553274, 0.9870580378486575, 0.9950909120315392, 0.9936504389502406, 0.9968750692520241, 0.9924735468266829]


## RandomForest

In [9]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 3,
            min_samples_split = 5,
            max_leaf_nodes = 5,
            min_samples_leaf =2,
            random_state = 42)  
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.004181618825684509
[0.002020761597096426, 0.0034743966616410924, 0.004331097545734795, 0.004048422384113505, 0.00642353672115828, 0.0032072181936474444, 0.008021583840430606, 0.005468102583731626, 0.001972294595174342, 0.002848774134116979]


test_r2_ave: 0.9817658418847554
[0.9937463934809763, 0.9903168084609834, 0.9777770873145517, 0.9781833959398453, 0.9814205955064246, 0.9880788510278739, 0.9557102282916768, 0.962341417621272, 0.9966866597562453, 0.9933969814477038]


In [10]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=2e-06)
elastic_mod=ElasticNet(alpha=[0.03], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 130, 
              max_depth = 1, 
              min_child_weight = 1,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.85,
              reg_alpha =  0,
              reg_lambda = 1)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 3,
            min_samples_split = 5,
            max_leaf_nodes = 5,
            min_samples_leaf =2,
            random_state = 42)  
vote_mod = VotingRegressor([ 
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])
# ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.003068623796616844
[0.0009483507776673911, 0.002611465466017467, 0.002586169176551354, 0.0025830216554112414, 0.005782034968448913, 0.003221636919656954, 0.004813690670460703, 0.003514926592105424, 0.0019372538954425817, 0.002687687844406412]


test_r2_ave: 0.9908681339728098
[0.9986226687499182, 0.9945294839240261, 0.992076466248943, 0.9911188008355096, 0.9849462466762671, 0.9879714220041925, 0.9840507764447718, 0.984439513213042, 0.996803346585923, 0.9941226150455046]


In [11]:

stack_mod = StackingRegressor(regressors=[elastic_mod,random_mod,lr, vote_mod], 
                           meta_regressor=xgb_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.002336604412747979
[0.0005606020904165517, 0.003085366434268515, 0.0006030647261745316, 0.0018276650727053762, 0.005383991030378199, 0.002508698087546982, 0.0025013639757019437, 0.0018433462174697714, 0.002227194494223421, 0.0028247519985944995]


test_r2_ave: 0.9947355518797021
[0.9995187066981751, 0.9923638715458833, 0.9995691432476526, 0.9955535978462734, 0.98694755046044, 0.9927061320097296, 0.9956933758697547, 0.9957203871328738, 0.995774882852216, 0.9935078711340227]


In [12]:
ave(skew_data[title],y,0.4,stack_mod,0.2,vote_mod,0.4,xgb_mod)

test_rmse_ave: 0.002396784809201697
[0.0003619935783621463, 0.002527960344355267, 0.0012071318611596785, 0.0017612975070955424, 0.005568665808043304, 0.0028039487635770632, 0.0027756865533178567, 0.0021726045728568052, 0.001962121059016123, 0.0028264380442331836]


test_r2_ave: 0.9944715308451544
[0.9997993209092387, 0.9948737446170883, 0.998273707135197, 0.9958706570269538, 0.9860367768610667, 0.9908882614765965, 0.9946969706517691, 0.9940549975357944, 0.9967207534957522, 0.9935001187420871]
